# Radiation fluxes in the LongRunMIP
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

#### Mechanics
Dependencies

In [12]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, cm
from matplotlib import rcParams
import matplotlib.gridspec as gridspec
import seaborn as sns
from scipy.io import loadmat
from matplotlib.patches import Polygon
from scipy import signal
from scipy import stats

Model names

In [13]:
modelnames_fig = ['CESM1.0.4 abrupt 2x','CESM1.0.4 abrupt 4x','CESM1.0.4 abrupt 8x', 'CNRM-CM6.1 abrupt4x', \
    'GFDL-CM3 1pct 2x','GFDL-ESM2M 1pct 2x','GISS-E2-R 1pct 4x', \
    'GISS-E2-R abrupt 4x','HadCM3L abrupt 2x','HadCM3L abrupt 4x', \
    'HadCM3L abrupt 6x','HadCM3L abrupt 8x','IPSL-CM5A-LR abrupt 4x', \
    'MIROC3.2 1pct 2x','MIROC3.2 1pct 4x','MPIESM-1.2 abrupt 2x', \
    'MPIESM-1.2 abrupt 4x','MPIESM-1.2 abrupt 8x']

modelnames_file = ['CESM104_abrupt2x','CESM104_abrupt4x','CESM104_abrupt8x', \
    'CNRMCM61_abrupt4x','GFDLCM3_1pct2x','GFDLESM2M_1pct2x','GISSE2R_1pct4x', \
    'GISSE2R_abrupt4x','HadCM3L_abrupt2x','HadCM3L_abrupt4x', \
    'HadCM3L_abrupt6x','HadCM3L_abrupt8x','IPSLCM5A_abrupt4x', \
    'MIROC32_1pct2x','MIROC32_1pct4x','MPIESM12_abrupt2x', \
    'MPIESM12_abrupt4x','MPIESM12_abrupt8x']

Location

In [14]:
loc_out = "../Data/RadiativeFlux/"

fluxes

In [15]:
fluxes = ["rsus","rsds","rlus","rlds"]

Indices

In [16]:
indices = ["nino3","nino34","nino4"]

lat_mins = [-5,-5,-5]
lat_maxs = [5,5,5]
lon_mins = [210,190,160]
lon_maxs = [270,240,210]

Custom detrending function

In [17]:
def custom_detrend(data,order):
    
    # only for one-dimensional timeseries
    # numpy required
    
    x = np.arange(1,len(data)+1,1)
    
    model = np.polyfit(x,data,order)
    predicted = np.polyval(model,x)
    new = data - predicted
    
    return(new)

Function for listing files

In [18]:
def list_files(directory, extension):
    
    # Credit to: http://www.martinbroadhurst.com/listing-all-files-in-a-directory-with-a-certain-extension-in-python.html
    # Requires os.listdir
    
    return (f for f in os.listdir(directory) if f.endswith(extension))

Detrending order

In [19]:
ordr = 2

#### Analysis

In [20]:
for i in np.arange(0,len(modelnames_file),1):
    
    model, exp = modelnames_file[i].split("_")
    print(modelnames_file[i])
    
    for ff in np.arange(0,len(fluxes),1):
        flux = fluxes[ff]
        print(flux)
        
        loc_flux = "~/"  # change if using raw data
        fname_flux_exp = [f for f in os.listdir(loc_flux) if f.startswith(flux+"_mon_"+modelnames_file[i])][0]
        fname_flux_control = [f for f in os.listdir(loc_flux) if f.startswith(flux+"_mon_"+model+"_control")][0]
    
        model_decode_times_list = ["MIROC32","MPIESM12"]

        if model in model_decode_times_list:
            dc = False
        else:
            dc = True

        flux_exp_global = xr.DataArray(xr.open_dataset(loc_flux+fname_flux_exp,decode_times=dc).data_vars[flux]) 
        flux_control_global = xr.DataArray(xr.open_dataset(loc_flux+fname_flux_control,decode_times=dc).data_vars[flux])
        
    
        if model == "HadCM3L":
            lat = flux_exp_global.coords["latitude_1"]
            lon = flux_exp_global.coords["longitude_1"]
            latname = "latitude_1"
            lonname = "longitude_1"
        else:
            lat = flux_exp_global.coords["lat"]
            lon = flux_exp_global.coords["lon"]
    
        # loop through nino indices
    
        for j in np.arange(0,len(indices),1):
            #print(indices[j])

            if lat[0] > lat[len(lat.values)-1]:
                latmax = lat_mins[j]
                latmin = lat_maxs[j]
            else:
                latmax = lat_maxs[j]
                latmin = lat_mins[j]
            
            # area-average
            flux_index_exp_raw = flux_exp_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
            flux_index_control_raw = flux_control_global.loc[:,latmin:latmax,lon_mins[j]:lon_maxs[j]].mean(axis=(1,2))
        
        
            # time coordinates
            n_months_c = flux_index_control_raw.shape[0]
            n_months_f = flux_index_exp_raw.shape[0]
            time_exp = xr.cftime_range(start='0001', periods=n_months_f, freq='M')
            time_control = xr.cftime_range(start='0001', periods=n_months_c, freq='M')

            # attach time coords to calculate anomalies
            flux_index_exp_time = xr.DataArray(flux_index_exp_raw.values,coords=[time_exp],dims=["time"])
            flux_index_control_time = xr.DataArray(flux_index_control_raw.values,coords=[time_control],dims=["time"])
            
            #print("detrending and calculating anomalies...")
            # detrend fast/transient/eq (if abrupt) and transient/eq (if 1pct)
            # then calculate anomalies

            # detrend control
            flux_index_ctrl_detrend = xr.DataArray(custom_detrend(flux_index_control_time,1).values,coords=[time_control],dims=["time"])
            flux_index_ctrl_anom = flux_index_ctrl_detrend.groupby("time.month") - (flux_index_ctrl_detrend.groupby("time.month").mean(dim="time"))
                
                
            # write out index from control
            flux_index_ctrl_anom.name = flux
            flux_index_ctrl_anom.attrs["creation_date"] = str(datetime.datetime.now())
            flux_index_ctrl_anom.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            flux_index_ctrl_anom.attrs["data_description"] = flux+" in "+indices[j]+" region  from "+model+" control, anomaly and linearly detrended"
            flux_index_ctrl_anom.attrs["created_from"] = "Calculate_RadiativeFlux_Indices.ipynb"

            fname_out_control_anom = loc_out+flux+"_"+indices[j]+"_"+model+"_control_anom_detrend.nc"
            flux_index_ctrl_anom.to_netcdf(fname_out_control_anom,mode="w")
            print(fname_out_control_anom)
            
            
            # conditional for abrupt/1pct
            ntimes_peryear = 12

            if exp.startswith("abrupt"):

                index_fast = flux_index_exp_time[0:(50*ntimes_peryear)]
                index_transient = flux_index_exp_time[(50*ntimes_peryear):(150*ntimes_peryear)]
                index_eq = flux_index_exp_time[(150*ntimes_peryear):]

                index_fast_detrend = xr.DataArray(custom_detrend(index_fast,ordr).values,coords=[index_fast.coords["time"]],dims=["time"])
                index_transient_detrend = xr.DataArray(custom_detrend(index_transient,ordr).values,coords=[index_transient.coords["time"]],dims=["time"])
                index_eq_detrend = xr.DataArray(custom_detrend(index_eq,ordr).values,coords=[index_eq.coords["time"]],dims=["time"])

                index_fast_anom = index_fast_detrend.groupby("time.month") - (index_fast_detrend.groupby("time.month").mean(dim="time"))
                index_transient_anom = index_transient_detrend.groupby("time.month") - (index_transient_detrend.groupby("time.month").mean(dim="time"))
                index_eq_anom = index_eq_detrend.groupby("time.month") - (index_eq_detrend.groupby("time.month").mean(dim="time"))

                index_exp_anom = xr.concat([index_fast_anom,index_transient_anom,index_eq_anom],dim="time")

            else:

                index_transient = flux_index_exp_time[0:(140*ntimes_peryear)]
                index_eq = flux_index_exp_time[(140*ntimes_peryear):]

                index_transient_detrend = xr.DataArray(custom_detrend(index_transient,ordr).values,coords=[index_transient.coords["time"]],dims=["time"])
                index_eq_detrend = xr.DataArray(custom_detrend(index_eq,ordr).values,coords=[index_eq.coords["time"]],dims=["time"])

                index_transient_anom = index_transient_detrend.groupby("time.month") - (index_transient_detrend.groupby("time.month").mean(dim="time"))
                index_eq_anom = index_eq_detrend.groupby("time.month") - (index_eq_detrend.groupby("time.month").mean(dim="time"))

                index_exp_anom = xr.concat([index_transient_anom,index_eq_anom],dim="time")

            # write out index from experiment
            index_exp_anom.name = flux
            index_exp_anom.attrs["creation_date"] = str(datetime.datetime.now())
            index_exp_anom.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
            index_exp_anom.attrs["data_description"] = flux+" in "+indices[j]+" region from "+model+" "+exp+", anomaly and detrended using order in file name"
            index_exp_anom.attrs["created_from"] = "Calculate_RadiativeFlux_Indices.ipynb"

            fname_out_exp_anom = loc_out+flux+"_"+indices[j]+"_"+model+"_"+exp+"_anom_detrend"+str(ordr)+".nc"
            index_exp_anom.to_netcdf(fname_out_exp_anom,mode="w")
            print(fname_out_exp_anom)

GISSE2R_1pct4x
rsus
../Data/RadiativeFlux/rsus_nino3_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsus_nino3_GISSE2R_1pct4x_anom_detrend2.nc
../Data/RadiativeFlux/rsus_nino34_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsus_nino34_GISSE2R_1pct4x_anom_detrend2.nc
../Data/RadiativeFlux/rsus_nino4_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsus_nino4_GISSE2R_1pct4x_anom_detrend2.nc
rsds
../Data/RadiativeFlux/rsds_nino3_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsds_nino3_GISSE2R_1pct4x_anom_detrend2.nc
../Data/RadiativeFlux/rsds_nino34_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsds_nino34_GISSE2R_1pct4x_anom_detrend2.nc
../Data/RadiativeFlux/rsds_nino4_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rsds_nino4_GISSE2R_1pct4x_anom_detrend2.nc
rlus
../Data/RadiativeFlux/rlus_nino3_GISSE2R_control_anom_detrend.nc
../Data/RadiativeFlux/rlus_nino3_GISSE2R_1pct4x_anom_detrend2.nc
../Data/RadiativeFlux/rlus_nino34_GISSE2R_control_anom_d